# Integration of membrane protein and transport of substrates sing BioCRNpyler membrane components and mechanism

Consider the following substrate transport steps:
1. Expression of membrane protein: G + EnergyTXTL $\rightarrow$ membrane_protein (MP)
2. Integration of membrane protein in membrane: MP $\rightarrow$ integral_membrane_protein (IMP)
3. Diffusion or transport across membrane 

Bringing in work horses and TX-TL mixture intending to use

In [1]:
#Import bioCRNplyer and plotting
from biocrnpyler import *
import numpy as np
import pylab as plt
%matplotlib inline
import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()
from bokeh.themes import Theme

# Modules needed from Bokeh.
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import LinearAxis, Range1d

colors = bokeh.palettes.Colorblind[8]

try:
    import dnaplotlib as dpl
    dpl_enabled = True
except (ModuleNotFoundError,ImportError) as e:
    dpl_enabled = False
    
theme = Theme(json={'attrs': {
# apply defaults to Figure properties
'Figure': {
    'toolbar_location': 'right',
    'outline_line_color': None,
    'min_border_right': 10,
#     'sizing_mode': 'stretch_width',
    'height':800,
    'width':800,
},'Grid': {
    'grid_line_color': None,
},
'Title': {
    'text_font_size': '20pt',
    'align': 'center'
},
    
    
# apply defaults to Axis properties
'Axis': {
#     'minor_tick_out': None,
#     'minor_tick_in': None,
    'major_label_text_font_size': '15pt',
    'axis_label_text_font_size': '15pt',
#     'axis_label_text_font': 'Work Sans',
    'axis_label_text_font_style': 'normal',
    'axis_label_standoff':15
},


# apply defaults to Legend properties
'Legend': {
    'background_fill_alpha': 0.8,
    'location': 'top_right',
    "label_text_font_size": '15pt'
}}})

bokeh.io.curdoc().theme = theme

Loading BokehJS ...

Activatable expression of arbitrary membrane protein:

In [2]:
#ActivatedPromoter
activator = Species("T7RNAP", material_type = "small_molecule")

#Create a custom set of parameters
hill_parameters = {"k":1.0, "n":4, "K":20, "kleak":0.0001}
#By Loading custom parameters into the promoter, we override the default parameters of the Mixture import numpy as np

P_activatable = ActivatablePromoter("P_activtable", activator = activator, leak = False, parameters = hill_parameters)

#Create a DNA assembly "reporter" with P_activatable for its promoter
activatable_assembly = DNAassembly("activatable_assembly", promoter = P_activatable, rbs = "rbs", initial_concentration = 1*10**-6, protein= "memProtein")

E = EnergyTxTlExtract( components=[activatable_assembly], parameter_file = "txtl_toolbox_parameters.txt")

CRN = E.compile_crn()

# print(CRN.pretty_print())

C:\Users\zoila\anaconda3\lib\site-packages\biocrnpyler-1.1.1-py3.8.egg\biocrnpyler\parameter.py:507: UserWarning: parameter file contains no unit column! Please add a column named ['unit', 'units'].


# Example 1: Integration of membrane protein (monomer) and passive diffusion by a membrane channel (ex. IPTG transport though $\alpha$-hemolysin)

In [3]:
memProtein = IntegralMembraneProtein('memProtein', product='mem_channel')

default_parameters = {"kd1":5,"kb1":2e-3, "ku1": 2e-10 ,"kcat":10, "kex":.5,"kd2":5 }
#create a catalysis mechanism. 
mech_cat = Membrane_Protein_Integration()

#place that mechanism in a dictionary: "catalysis":mech_cat
default_mechanisms = {mech_cat.mechanism_type:mech_cat}

M1= Mixture("Default Param Pathway", components = [memProtein], parameters = default_parameters, mechanisms = default_mechanisms)
print("repr(Mixture) gives a printout of what is in a mixture and what it's Mechanisms are:\n", repr(M1),"\n")

#Compile the CRN with Mixture.compile_crn
CRN1 = M1.compile_crn()
print(CRN1.pretty_print(show_rates = True, show_attributes = True, show_materials = True, show_keys = False))

repr(Mixture) gives a printout of what is in a mixture and what it's Mechanisms are:
 Mixture: Default Param Pathway
Components = [
	IntegralMembraneProtein: memProtein ]
Mechanisms = {
	catalysis:membrane_protein_integration } 

IntegralMembraneProtein: memProtein
Species(N = 2) = {
Passive[mem_channel] (@ 0),  protein[memProtein] (@ 0),  
}

Reactions (2) = [
0. protein[memProtein] --> 
 Kf=k_forward * protein_memProtein
  k_forward=5

1. protein[memProtein] --> Passive[mem_channel]
 Kf = k protein[memProtein] / ( 1 + (Passive[mem_channel]/K)^4 )
  k=0.5
  K=10
  n=4

]


### Define membrane channel and transport mechanism

In [19]:
Mem_channel=MembraneChannel(memProtein.product, substrate='Sub')

default_parameters = {"kb1":1e-1}

mech_cat = Passive_Transport()

default_mechanisms = {mech_cat.mechanism_type:mech_cat}

M2 = Mixture("Default Param Pathway", components = [Mem_channel], parameters = default_parameters, mechanisms = default_mechanisms)
print("repr(Mixture) gives a printout of what is in a mixture and what it's Mechanisms are:\n", repr(M2),"\n")

#Compile the CRN with Mixture.compile_crn
CRN2 = M2.compile_crn()
print(CRN2.pretty_print(show_rates = True, show_attributes = True, show_materials = True, show_keys = False))

[]
repr(Mixture) gives a printout of what is in a mixture and what it's Mechanisms are:
 Mixture: Default Param Pathway
Components = [
	MembraneChannel: mem_channel ]
Mechanisms = {
	catalysis:passive_membrane_protein_transport } 

Species(N = 3) = {
Passive[mem_channel] (@ 0),  Sub (@ 0),  Sub (@ 0),  
}

Reactions (1) = [
0. Sub+Passive[mem_channel] <--> Sub+Passive[mem_channel]
 Kf=k_forward * Sub_Internal * Passive_mem_channel
 Kr=k_reverse * Sub_External * Passive_mem_channel
  k_forward=0.1
  k_reverse=0.1

]


#### Combine all mixtures

In [20]:
addReactions=CRN1.reactions + CRN2.reactions
addSpecies=CRN1.species + CRN2.species

CRN.add_species(addSpecies)
CRN.add_reactions(addReactions)

print("Writing the CRN model as an SBML file (to output, exchange, standardize the model)...")
CRN.write_sbml_file('PassiveMemChannel_Test.xml');
print("Done!")
print(CRN.pretty_print(show_rates = True, show_attributes = True, show_materials = True, show_keys = False))

Writing the CRN model as an SBML file (to output, exchange, standardize the model)...
Done!
Species(N = 17) = {
metabolite[amino_acids] (@ 30.0),  metabolite[Fuel_3PGA] (@ 30.0),  metabolite[NTPs] (@ 5.0),  protein[Ribo] (@ 0.0273),  protein[RNAP] (@ 0.00933),  dna[activatable_assembly] (@ 1e-06),  complex[protein[Ribo]:rna[activatable_assembly]] (@ 0),  complex[protein[RNAase]:rna[activatable_assembly]] (@ 0),  Passive[mem_channel] (@ 0),  protein[memProtein] (@ 0),  complex[complex[protein[Ribo]:rna[activatable_assembly]]:protein[RNAase]] (@ 0),  rna[activatable_assembly] (@ 0),  small_molecule[T7RNAP] (@ 0),  Sub (@ 0),  Sub (@ 0),  protein[RNAase] (@ 0),  metabolite[NDPs] (@ 0),  
}

Reactions (14) = [
0. dna[activatable_assembly] --> dna[activatable_assembly]+rna[activatable_assembly]
 Kf = k dna[activatable_assembly] small_molecule[T7RNAP]^n / ( K^n + small_molecule[T7RNAP]^n )
  k=1.0
  K=20
  n=4

1. rna[activatable_assembly]+protein[Ribo] <--> complex[protein[Ribo]:rna[activat

### Simulating CRN

In [26]:
#Inital conditions
x0_dict= {'small_molecule_T7RNAP':8, 'Sub_Internal':0.75,'Sub_External':0.25,}

# Simulate CRN
maxtime = 30000
timepoints = np.arange(0, maxtime, 100)

#Run Simulation
R = CRN.simulate_with_bioscrape_via_sbml(timepoints = timepoints, initial_condition_dict = x0_dict)

#Plot
p1 = bokeh.plotting.figure()
p1.title.text = "Passive Transport of Substrate"
p1.xaxis.axis_label = 'Time (hours)'
p1.yaxis.axis_label = 'Substrate Conc. (uM)'

p1.line(R["time"]/3600,  R['Sub_External'], line_width = 3, legend_label = "External_substrate", color = colors[2])
p1.line(R["time"]/3600,  R['Sub_Internal'], line_width = 3, legend_label = "Internal_substrate", color = colors[3], line_dash = 'dashed')
        
bokeh.io.show(p1)

# Example 2: Integration of membrane protein (homdimer) and facilitated passive diffusion by a membrane channel (ex. )

In [9]:
memProtein = IntegralMembraneProtein('memProtein', product='Mem_channel', direction='Exporter', size= 2)

default_parameters = {"kd1":5,"kb1":2e-3, "ku1": 2e-10 ,"kcat":10, "kex":.5,"kd2":5 }
#creeate a catalysis mechanism. 
mech_cat = Membrane_Protein_Integration()

#place that mechanism in a dictionary: "catalysis":mech_cat
default_mechanisms = {mech_cat.mechanism_type:mech_cat}

M1= Mixture("Default Param Pathway", components = [memProtein], parameters = default_parameters, mechanisms = default_mechanisms)
print("repr(Mixture) gives a printout of what is in a mixture and what it's Mechanisms are:\n", repr(M1),"\n")

#Compile the CRN with Mixture.compile_crn
CRN1 = M1.compile_crn()
print(CRN1.pretty_print(show_rates = True, show_attributes = True, show_materials = True, show_keys = False))

repr(Mixture) gives a printout of what is in a mixture and what it's Mechanisms are:
 Mixture: Default Param Pathway
Components = [
	IntegralMembraneProtein: memProtein ]
Mechanisms = {
	catalysis:membrane_protein_integration } 

IntegralMembraneProtein: memProtein
Species(N = 3) = {
complex[2x_protein[memProtein]] (@ 0),  protein[memProtein] (@ 0),  Exporter[Mem_channel] (@ 0),  
}

Reactions (4) = [
0. protein[memProtein] --> 
 Kf=k_forward * protein_memProtein
  k_forward=5

1. 2protein[memProtein] <--> complex[2x_protein[memProtein]]
 Kf=k_forward * protein_memProtein^2
 Kr=k_reverse * complex_protein_memProtein_2x_
  k_forward=0.002
  k_reverse=2e-10

2. complex[2x_protein[memProtein]] --> 
 Kf=k_forward * complex_protein_memProtein_2x_
  k_forward=5

3. complex[2x_protein[memProtein]] --> Exporter[Mem_channel]
 Kf = k complex[2x_protein[memProtein]] / ( 1 + (Exporter[Mem_channel]/K)^4 )
  k=0.5
  K=10
  n=4

]


### Define membrane channel and transport mechanism

In [10]:
Mem_channel=MembraneChannel(memProtein.product, substrate='Sub', direction='Exporter')

default_parameters = {"kb1":1e-2, "ku2":1e-1}

mech_cat = Facilitated_Passive_Transport()

default_mechanisms = {mech_cat.mechanism_type:mech_cat}

M2 = Mixture("Default Param Pathway", components = [Mem_channel], parameters = default_parameters, mechanisms = default_mechanisms)
print("repr(Mixture) gives a printout of what is in a mixture and what it's Mechanisms are:\n", repr(M2),"\n")

#Compile the CRN with Mixture.compile_crn
CRN2 = M2.compile_crn()
print(CRN2.pretty_print(show_rates = True, show_attributes = True, show_materials = True, show_keys = False))

[]
repr(Mixture) gives a printout of what is in a mixture and what it's Mechanisms are:
 Mixture: Default Param Pathway
Components = [
	MembraneChannel: Mem_channel ]
Mechanisms = {
	catalysis:passive_membrane_protein_transport } 

Species(N = 5) = {
Sub (@ 0),  Sub (@ 0),  Exporter[Mem_channel] (@ 0),  complex[Exporter[Mem_channel]:Sub] (@ 0),  complex[Exporter[Mem_channel]:Sub] (@ 0),  
}

Reactions (4) = [
0. Sub+Exporter[Mem_channel] --> complex[Exporter[Mem_channel]:Sub]
k1*Sub_Internal*Exporter_Mem_channel-k1*Sub_External*Exporter_Mem_channel
  k1=0.1

1. complex[Exporter[Mem_channel]:Sub] --> Exporter[Mem_channel]+Sub
 Kf=k_forward * complex_Exporter_Mem_channel_Sub_Internal_
  k_forward=0.1

2. complex[Exporter[Mem_channel]:Sub] --> complex[Exporter[Mem_channel]:Sub]
 Kf=k_forward * complex_Exporter_Mem_channel_Sub_Internal_
  k_forward=0.01

3. complex[Exporter[Mem_channel]:Sub] --> Sub+Exporter[Mem_channel]
 Kf=k_forward * complex_Exporter_Mem_channel_Sub_External_
  k_forwar

#### Combine all mixtures

In [11]:
addReactions=CRN1.reactions + CRN2.reactions
addSpecies=CRN1.species + CRN2.species

CRN.add_species(addSpecies)
CRN.add_reactions(addReactions)

print("Writing the CRN model as an SBML file (to output, exchange, standardize the model)...")
CRN.write_sbml_file('ExporterMemCarrier_Test.xml');
print("Done!")
print(CRN.pretty_print(show_rates = True, show_attributes = True, show_materials = True, show_keys = False))

Writing the CRN model as an SBML file (to output, exchange, standardize the model)...
Done!
Species(N = 20) = {
metabolite[amino_acids] (@ 30.0),  metabolite[Fuel_3PGA] (@ 30.0),  metabolite[NTPs] (@ 5.0),  protein[Ribo] (@ 0.0273),  protein[RNAP] (@ 0.00933),  dna[activatable_assembly] (@ 1e-06),  complex[2x_protein[memProtein]] (@ 0),  complex[protein[Ribo]:rna[activatable_assembly]] (@ 0),  complex[protein[RNAase]:rna[activatable_assembly]] (@ 0),  protein[memProtein] (@ 0),  complex[complex[protein[Ribo]:rna[activatable_assembly]]:protein[RNAase]] (@ 0),  rna[activatable_assembly] (@ 0),  small_molecule[T7RNAP] (@ 0),  Sub (@ 0),  Sub (@ 0),  protein[RNAase] (@ 0),  metabolite[NDPs] (@ 0),  Exporter[Mem_channel] (@ 0),  complex[Exporter[Mem_channel]:Sub] (@ 0),  complex[Exporter[Mem_channel]:Sub] (@ 0),  
}

Reactions (24) = [
0. dna[activatable_assembly] --> dna[activatable_assembly]+rna[activatable_assembly]
 Kf = k dna[activatable_assembly] small_molecule[T7RNAP]^n / ( K^n + sma

### Simulating CRN

In [12]:
#Inital conditions
x0_dict= {'small_molecule_T7RNAP':8, 'Sub_Internal':0.75,'Sub_External':0.25,}

# Simulate CRN
maxtime = 30000
timepoints = np.arange(0, maxtime, 100)

#Run Simulation
R = CRN.simulate_with_bioscrape_via_sbml(timepoints = timepoints, initial_condition_dict = x0_dict)

p1 = bokeh.plotting.figure()
p1.title.text = "Facilitated Passive Transport of Substrate"
p1.xaxis.axis_label = 'Time (hours)'
p1.yaxis.axis_label = 'Substrate Conc. (uM)'

p1.line(R["time"]/3600,  R['Sub_External'], line_width = 3, legend_label = "External_substrate", color = colors[2])
p1.line(R["time"]/3600,  R['Sub_Internal'], line_width = 3, legend_label = "Internal_substrate", color = colors[3], line_dash = 'dashed')
        
bokeh.io.show(p1)

# Example 3: Integration of membrane protein (homodimer) and active transport by a membrane channel (ex. antibiotic transport by MsbA)

In [3]:
memProtein = IntegralMembraneProtein('memProtein', product='Mem_pump', direction='Exporter')

default_parameters = {"kd1":5,"kb1":2e-3, "ku1": 2e-10 ,"kcat":10, "kex":.5,"kd2":5 }
#creeate a catalysis mechanism. 
mech_cat = Membrane_Protein_Integration()

#place that mechanism in a dictionary: "catalysis":mech_cat
default_mechanisms = {mech_cat.mechanism_type:mech_cat}

M1= Mixture("Default Param Pathway", components = [memProtein], parameters = default_parameters, mechanisms = default_mechanisms)
print("repr(Mixture) gives a printout of what is in a mixture and what it's Mechanisms are:\n", repr(M1),"\n")

#Compile the CRN with Mixture.compile_crn
CRN1 = M1.compile_crn()
print(CRN1.pretty_print(show_rates = True, show_attributes = True, show_materials = True, show_keys = False))

repr(Mixture) gives a printout of what is in a mixture and what it's Mechanisms are:
 Mixture: Default Param Pathway
Components = [
	IntegralMembraneProtein: memProtein ]
Mechanisms = {
	catalysis:membrane_protein_integration } 

IntegralMembraneProtein: memProtein
Species(N = 2) = {
protein[memProtein] (@ 0),  Exporter[Mem_pump] (@ 0),  
}

Reactions (2) = [
0. protein[memProtein] --> 
 Kf=k_forward * protein_memProtein
  k_forward=5

1. protein[memProtein] --> Exporter[Mem_pump]
 Kf = k protein[memProtein] / ( 1 + (Exporter[Mem_pump]/K)^4 )
  k=0.5
  K=10
  n=4

]


### Define membrane channel and transport mechanism

In [4]:
Mem_pump=MembranePump(memProtein.product, substrate='Sub', ATP=2)

default_parameters = {"kb1":1e0, "ku1":1e-1, "kb2":1e1, "ku2":2e-2, 
                      "kb3":1.5, "ku3":1.5, "kb4":1e1, "ku4":2e-2, 
                      "ku5":2, "ku6":1.5, 
                      "kex":.3,"kcat":0.1}

mech_cat = Primary_Active_Transport()

default_mechanisms = {mech_cat.mechanism_type:mech_cat}

M2= Mixture("Default Param Pathway", components = [Mem_pump], parameters = default_parameters, mechanisms = default_mechanisms)
print("repr(Mixture) gives a printout of what is in a mixture and what it's Mechanisms are:\n", repr(M2),"\n")

#Compile the CRN with Mixture.compile_crn
CRN2 = M2.compile_crn()
print(CRN2.pretty_print(show_rates = True, show_attributes = True, show_materials = True, show_keys = False))

[]
repr(Mixture) gives a printout of what is in a mixture and what it's Mechanisms are:
 Mixture: Default Param Pathway
Components = [
	MembranePump: Mem_pump ]
Mechanisms = {
	catalysis:passive_membrane_protein_transport } 

Species(N = 10) = {
complex[complex[Exporter[Mem_pump]:Sub]:2x_small_molecule[ATP]] (@ 0),  complex[complex[Exporter[Mem_pump]:Sub]:2x_small_molecule[ADP]] (@ 0),  Sub (@ 0),  Sub (@ 0),  Exporter[Mem_pump] (@ 0),  complex[Exporter[Mem_pump]:2x_small_molecule[ATP]] (@ 0),  complex[Exporter[Mem_pump]:Sub] (@ 0),  complex[Exporter[Mem_pump]:Sub] (@ 0),  small_molecule[ATP] (@ 0),  small_molecule[ADP] (@ 0),  
}

Reactions (7) = [
0. Sub+Exporter[Mem_pump] <--> complex[Exporter[Mem_pump]:Sub]
 Kf=k_forward * Sub_Internal * Exporter_Mem_pump
 Kr=k_reverse * complex_Exporter_Mem_pump_Sub_Internal_
  k_forward=1.0
  k_reverse=0.1

1. complex[Exporter[Mem_pump]:Sub]+2small_molecule[ATP] <--> complex[complex[Exporter[Mem_pump]:Sub]:2x_small_molecule[ATP]]
 Kf=k_forward * 

### Combine all mixtures

In [5]:
addReactions=CRN1.reactions + CRN2.reactions
addSpecies=CRN1.species + CRN2.species

CRN.add_species(addSpecies)
CRN.add_reactions(addReactions)

print("Writing the CRN model as an SBML file (to output, exchange, standardize the model)...")
CRN.write_sbml_file('ExporterMemTransporter_Test.xml');
print("Done!")
print(CRN.pretty_print(show_rates = True, show_attributes = True, show_materials = True, show_keys = False))

Writing the CRN model as an SBML file (to output, exchange, standardize the model)...
Done!
Species(N = 24) = {
metabolite[amino_acids] (@ 30.0),  metabolite[Fuel_3PGA] (@ 30.0),  metabolite[NTPs] (@ 5.0),  protein[Ribo] (@ 0.0273),  protein[RNAP] (@ 0.00933),  dna[activatable_assembly] (@ 1e-06),  complex[protein[Ribo]:rna[activatable_assembly]] (@ 0),  complex[protein[RNAase]:rna[activatable_assembly]] (@ 0),  protein[memProtein] (@ 0),  complex[complex[protein[Ribo]:rna[activatable_assembly]]:protein[RNAase]] (@ 0),  complex[complex[Exporter[Mem_pump]:Sub]:2x_small_molecule[ATP]] (@ 0),  complex[complex[Exporter[Mem_pump]:Sub]:2x_small_molecule[ADP]] (@ 0),  rna[activatable_assembly] (@ 0),  small_molecule[T7RNAP] (@ 0),  Sub (@ 0),  Sub (@ 0),  protein[RNAase] (@ 0),  metabolite[NDPs] (@ 0),  Exporter[Mem_pump] (@ 0),  complex[Exporter[Mem_pump]:2x_small_molecule[ATP]] (@ 0),  complex[Exporter[Mem_pump]:Sub] (@ 0),  complex[Exporter[Mem_pump]:Sub] (@ 0),  small_molecule[ATP] (@ 0),

### Simulating CRN

In [6]:
# Simulate CRN
maxtime = 30000
timepoints = np.arange(0, maxtime, 100)

line_style = ['solid', 'dashed', 'dotted', 'dotdash', 'dashdot', np.array([10])]
atp_conc = np.array([0, 2, 5, 10, 50]) #mM

p = bokeh.plotting.figure()
p.title.text = "Substrate Transport by Pump"
p.xaxis.axis_label = 'Time (hours)'
p.yaxis.axis_label = 'Substrate Conc. (uM)'

for index2, ATP in enumerate(atp_conc):
    x0_dict= {'small_molecule_T7RNAP':8, 'Sub_Internal':0.75,'Sub_External':0.25,'small_molecule_ATP_Internal':ATP}
    #Run Simulation
    R = CRN.simulate_with_bioscrape_via_sbml(timepoints = timepoints, initial_condition_dict = x0_dict)

    p.line(R["time"]/3600,  R['Sub_External'], line_width = 3, legend_label = "External_substrate", color = colors[index2])
    p.line(R["time"]/3600,  R['Sub_Internal'], line_width = 3, legend_label = "Internal_substrate", color = colors[index2],  line_dash = line_style[1])
#     p.line(R["time"]/3600,  R['small_molecule_ATP_Internal'], line_width = 3, legend_label = "ATP", color = colors[3],  line_dash = line_style[index2])
    p.legend.location = "top_left"
bokeh.io.show(p)